# SSB Kayıt 


## Step 1: Waiting for the Next Period Reading
- **State Number**: 1
- **State Code**: -
- **State Name**: Waiting for the Next Period Reading
- **Description**: This step waits for the periodic reading after the SSB registration date to trigger the creation of accrual. The system listens for an event (`thor-billing-accrual-created`) to confirm the accrual data and proceeds based on this information.
- **Status**: Pending

#### 1a. Çalışma öncesi Script

#### 1b. İstek Eşleştirme Scripti

#### 1c. State Context Değer Tutma 

#### 1d. Çalışma Sonrası Script

#### 1e. İş Mantığı

## Step 2: Contract Terminated
- **State Number**: 2
- **State Code**: -
- **State Name**: Contract Terminated
- **Description**: This step ensures that the contract is terminated under specific rules for different subscriber groups. If the contract is not renewed, it terminates based on the subscriber group being "Industrial." Public and private service sectors contracts are also considered for termination.
- **Status**: Done

#### 2a. Çalışma Öncesi Scripti

In [ ]:
# machine                     : Currently executing machine instance
# state                       : Currently executing state instance
# request_data                : Original trigger data
# context                     : Machine stored context (1. Muhatap(ID), 2. Tesisat(ID), 3. Sözleşme)
# state_responses_of_machine  : State responses of Machine <StateCode, {prev, current, webhook}>
# return
#     json input of business logic service
import json
from datetime import datetime
def state_handler(machine, state, request_data, context, state_response_of_machine):
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'

    
    subscriberGroup = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='SubscriberGroup'),None)['values'][0]['value']
    ssbDocEndDate = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='SsbDocumentEndDate'),None)['value']


    
    if request_data['isNew'] == True and subscriberGroup == 'SA':
        if ssbDocEndDate > current_date:
            return {"success": True, "goto": "10340003"}
        else:
            return {"success": True, "error": {"code": "END", "detail": "SsbDocumentEndDate"}}

    elif request_data['isNew'] == False and subscriberGroup == 'KO':
        return {"success": True, "goto": "10340003"}
    else :
        return {"success": True, "error": {"code": "END", "detail": "Subscriber Group"}}



#### 2b. İstek Eşleştirme Scripti

In [5]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine) -> json:  
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return json.dumps(data, indent=4)


#### 2c. State Context Değer Tutma Scripti

#### 2d. Çalışma Sonrası Scripti

## Step 3: Data Preparation for New Account
- **State Number**: 3
- **State Code**: -
- **State Name**: Data Preparation for New Account
- **Description**: In this step, the system prepares the data needed to create a new account. The necessary attribute values are gathered from the terminated contract and associated account details.
- **Status**: Pending

#### 3a. Çalışma Öncesi Script

#### 3b. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine) -> json:  
    contractAccountId = next((attr for attr in request_data['attributes'] if attr['attribute']['attributeCode']=='ContractAccountId'),None)['value']

    return {"assetid":contractAccountId}

#### 3c. State Context Değer Tutma Scripti

#### 3d. Çalışma Sonrası Scripti 

## Step 4: New Account Created
- **State Number**: 4
- **State Code**: -
- **State Name**: New Account Created
- **Description**: This step creates a new account based on the details of the terminated contract. Some attribute values may differ, and these changes are reflected in the new account. Historical data for the old account is also updated to match the new expiry dates.
- **Status**: Pending


#### 4A. Çalışma Öncesi Scripti

#### 4B. İstek Eşleştirme Scripti

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, state_response_of_machine):

    prev_state_response = state_response_of_machine
    
    create_date = next((attr for attr in prev_state_response['attributes'] if attr['attribute']['attributeCode'] == 'CreateDate'),None)['value'] 
    
    AccountClassCode = 1
    TaxClass = 2
    AccountGLCode = 3
    AccountSubscriberGroup = 4
    
    changes_dict = {'AccountClassCode': AccountClassCode,
                    'TaxClass': TaxClass,
                    'AccountGLCode': AccountGLCode,
                    'ExpiryDate':create_date
                    }
    
    
    if AccountSubscriberGroup == 'IndustrySubscriberGroup':
        changes_dict['AccountSubscriberGroup'] = 'PAPSSAOSG'
    else:
        changes_dict['AccountSubscriberGroup'] = 'IndustrySubscriberGroup'
    
    copied_json = {}
    stack = [(copied_json, prev_state_response)]

    # Copy context to copied_json without id
    while stack:
        current_copied, current_original = stack.pop()

        for key, value in current_original.items():
            if key != 'id':
                if isinstance(value, list):
                    new_list = []
                    for item in value:
                        if isinstance(item, dict):
                            new_item = {}
                            new_list.append(new_item)
                            stack.append((new_item, item))
                        else:
                            new_list.append(item)
                    current_copied[key] = new_list
                elif isinstance(value, dict):
                    new_dict = {}
                    current_copied[key] = new_dict
                    stack.append((new_dict, value))
                else:
                    current_copied[key] = value
    
    for code, new_value in changes_dict.items():
        for attr in copied_json.get('attributes', []):
            if attr['attribute']['attributeCode'] == code:
                if attr['value'] is None:
                    attr['values'][0]['value'] = new_value
                    attr['values'][0]['valueDesc'] = None
                else:
                    attr['values'][0]['valueDesc'] = None
                    attr['value'] = new_value
    
    return copied_json


#### 4C. State Context Değer Tutma Scripti

#### 4D. Çalışma Sonrası Scripti

#### 4E. İş Mantığı

In [6]:
"""md/save"""

'md/save'

## Step 5: New Contract Created
- **State Number**: 5
- **State Code**: -
- **State Name**: New Contract Created
- **Description**: A new contract is created using the data from the terminated contract. If the contract was not renewed, this step ensures the creation of a contract for the "Industrial" subscriber group. For public and private service sectors, the contract is generated with new attributes like ContractAccountId, SubscriberGroup, and ContractBeginDate.
- **Status**: Pending

#### 5A. Çalışma Öncesi Scripti

#### 5B. İstek Eşleştirme Scripti

In [ ]:
def state_handler(machine, state, request_data, context, state_response_of_machine):
    